In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PLUME_function import *

%matplotlib inline

from sklearn import linear_model

## Data importing

In [198]:
X_train_train = pd.read_csv('sample data/X_train_train.csv')
X_train_test = pd.read_csv('sample data/X_train_test.csv')
Y_train_train = pd.read_csv('sample data/Y_train_train.csv')
Y_train_test = pd.read_csv('sample data/X_train_test.csv')

In [823]:
Xtrain,Ytrain,Xtest,VarNames = DataImporting()

In [1114]:
var = 'O3-wind-is-day-month'
objet_predict = "O3"
X_train = pd.get_dummies(DataSample(X_train_train,var)).values
#X_train = X_train_train.iloc[:,1:]
Y_train = DataSample(Y_train_train,objet_predict).values

X_test = pd.get_dummies(DataSample(X_train_test,var)).values
#X_test = X_train_test.iloc[:,1:]
Y_test = DataSample(Y_train_test,objet_predict).values



In [1115]:
X_train.shape

(3011, 2769)

## MLP (fully connected)

In [1116]:
def tanh(x):
    return np.tanh(x)

def tanh_deriv(x):
    return 1.0 - np.tanh(x)**2

def logistic(x):
    return 1/(1 + np.exp(-x))

def logistic_derivative(x):
    return 1*logistic(x)*(1-logistic(x))
def linear(x):
    return x/300.0
def linear_deriv(x):
    return np.float64(1/300.0)

class NeuralNetwork:
    def __init__(self, layers, activation='tanh'):
        """
        :param layers: A list containing the number of units in each layer.
        Should be at least two values
        :param activation: The activation function to be used. Can be
        "logistic" or "tanh"
        """
        if activation == 'logistic':
            self.activation = logistic
            self.activation_deriv = logistic_derivative
        elif activation == 'tanh':
            self.activation = tanh
            self.activation_deriv = tanh_deriv
        elif activation == 'linear':
            self.activation = linear
            self.activation_deriv = linear_deriv
        self.trace = []
        self.weights = []
        for i in range(1, len(layers) - 1):
            self.weights.append((2*np.random.random((layers[i - 1] + 1, layers[i]
                                + 1))-1)*0.25)
        self.weights.append((2*np.random.random((layers[i] + 1, layers[i +
                            1]))-1)*0.25)
    def fit(self, X, y, learning_rate=0.2, epochs=1000):
        X = np.atleast_2d(X)
        temp = np.ones([X.shape[0], X.shape[1]+1])
        temp[:, 0:-1] = X  # adding the bias unit to the input layer
        X = temp
        y = np.array(y)

        for k in range(epochs):
            i = np.random.randint(X.shape[0])
            a = [X[i]]

            for l in range(len(self.weights)):
                a.append(self.activation(np.dot(a[l], self.weights[l])))
            error = y[i] - a[-1]
            deltas = [error * self.activation_deriv(a[-1])]
            self.trace.append(error)
            #print k,error
           

            for l in range(len(a) - 2, 0, -1): # we need to begin at the second to last layer
                deltas.append(deltas[-1].dot(self.weights[l].T)*self.activation_deriv(a[l]))
            deltas.reverse()
            for i in range(len(self.weights)):
                layer = np.atleast_2d(a[i])
                delta = np.atleast_2d(deltas[i])
                self.weights[i] += learning_rate * layer.T.dot(delta)
    def predict(self, x):
        x = np.array(x)
        temp = np.ones(x.shape[0]+1)
        temp[0:-1] = x
        a = temp
        for l in range(0, len(self.weights)):
            a = self.activation(np.dot(a, self.weights[l]))

        return a

In [1132]:
hour = 2
Y_hour_train = Y_train[:,hour-1]
Y_hour_test = Y_test[:,hour-1]

In [ ]:
nn = NeuralNetwork([np.shape(X_train)[1],4,1], 'linear')


nn.fit(X_train, Y_hour_train,learning_rate=0.003,epochs=61000)


MSE = 0 

for i in range(len(Y_hour_test)):
    MSE += (Y_hour_test[i] - nn.predict(X_test[i])) ** 2
print 'MSE(MLP) :', MSE*24


In [ ]:
plt.figure(figsize = [15,5])
plt.plot(Y_hour_test)
plt.plot([nn.predict(X_test[id]) for id in range(len(Y_hour_test))])

In [ ]:
plt.figure(figsize=[15,5])
plt.plot(nn.trace[:])

# The Final Model

## For NO2

In [1101]:
MSE_NO2 = 0 


var = 'NO2-is-day-month-windB-XPrecipIntensity_04143-XPrecipProbability_04143'
objet_predict = "NO2"

X_train = pd.get_dummies(DataSample(X_train_train,var)).values
#X_train = X_train_train.iloc[:,1:]
Y_train = DataSample(Y_train_train,objet_predict).values

X_test = pd.get_dummies(DataSample(X_train_test,var)).values
#X_test = X_train_test.iloc[:,1:]
Y_test = DataSample(Y_train_test,objet_predict).values


for hour in range(24):
    Y_hour_train = Y_train[:,hour]
    Y_hour_test = Y_test[:,hour]

    nn = NeuralNetwork([np.shape(X_train)[1],10,1], 'linear')
    nn.fit(X_train, Y_hour_train,learning_rate=0.03,epochs=6000)

    for i in range(len(Y_hour_test)):
        MSE_NO2 += (Y_hour_test[i] - nn.predict(X_test[i])) ** 2


MSE_NO2

array([ 4767401.5538634])

## For O3

In [1100]:
MSE_O3 = 0 


var = 'O3-is-windB-day-month'
objet_predict = "O3"

X_train = pd.get_dummies(DataSample(X_train_train,var)).values
#X_train = X_train_train.iloc[:,1:]
Y_train = DataSample(Y_train_train,objet_predict).values

X_test = pd.get_dummies(DataSample(X_train_test,var)).values
#X_test = X_train_test.iloc[:,1:]
Y_test = DataSample(Y_train_test,objet_predict).values


for hour in range(24):
    Y_hour_train = Y_train[:,hour]
    Y_hour_test = Y_test[:,hour]

    nn = NeuralNetwork([np.shape(X_train)[1],2,1], 'linear')
    nn.fit(X_train, Y_hour_train,learning_rate=0.03,epochs=6000)

    for i in range(len(Y_hour_test)):
        MSE_O3 += (Y_hour_test[i] - nn.predict(X_test[i])) ** 2



MSE_O3

array([ 9453779.31946302])

## For PM2.5

In [1102]:
MSE_PM2_5 = 0 


var = 'PM2-is-day-month-windB'
objet_predict = "PM2"

X_train = pd.get_dummies(DataSample(X_train_train,var)).values
#X_train = X_train_train.iloc[:,1:]
Y_train = DataSample(Y_train_train,objet_predict).values

X_test = pd.get_dummies(DataSample(X_train_test,var)).values
#X_test = X_train_test.iloc[:,1:]
Y_test = DataSample(Y_train_test,objet_predict).values


for hour in range(24):
    Y_hour_train = Y_train[:,hour]
    Y_hour_test = Y_test[:,hour]

    nn = NeuralNetwork([np.shape(X_train)[1],2,1], 'linear')
    nn.fit(X_train, Y_hour_train,learning_rate=0.05,epochs=4000)

    for i in range(len(Y_hour_test)):
        MSE_PM2_5 += (Y_hour_test[i] - nn.predict(X_test[i])) ** 2




MSE_PM2_5

array([ 2097886.95129554])

## For PM10

In [1103]:
MSE_PM10 = 0 


var = 'PM10-is-day-month-windB'
objet_predict = "PM10"

X_train = pd.get_dummies(DataSample(X_train_train,var)).values
#X_train = X_train_train.iloc[:,1:]
Y_train = DataSample(Y_train_train,objet_predict).values

X_test = pd.get_dummies(DataSample(X_train_test,var)).values
#X_test = X_train_test.iloc[:,1:]
Y_test = DataSample(Y_train_test,objet_predict).values


for hour in range(24):
    Y_hour_train = Y_train[:,hour]
    Y_hour_test = Y_test[:,hour]

    nn = NeuralNetwork([np.shape(X_train)[1],2,1], 'linear')
    nn.fit(X_train, Y_hour_train,learning_rate=0.05,epochs=4000)

    for i in range(len(Y_hour_test)):
        MSE_PM10 += (Y_hour_test[i] - nn.predict(X_test[i])) ** 2





MSE_PM10

array([ 1116261.88596783])

## Conclusion

In [1082]:
MSE = MSE_NO2 + MSE_O3 + MSE_PM2_5 + MSE_PM10
MSE = MSE/(float(len(Y_hour_test)))
print "MSE:",MSE

MSE: [ 16691.69068282]


In [ ]:
(float(len(Y_hour_test)))

In [840]:
test3 = pd.concat([pd.DataFrame([[1,2,3],[1,1,1]]), pd.DataFrame([[4,5,6],[2,2,2]])], axis=1)
test3.columns =Ytrain.columns.values[0:6]
test3

,PM2_5_04143_1,PM2_5_04143_2,PM2_5_04143_3,PM2_5_04143_4,PM2_5_04143_5,PM2_5_04143_6
0,1,2,3,4,5,6
1,1,1,1,2,2,2


In [836]:
Ytrain.columns.values

array(['PM2_5_04143_1', 'PM2_5_04143_2', 'PM2_5_04143_3', 'PM2_5_04143_4',
       'PM2_5_04143_5', 'PM2_5_04143_6', 'PM2_5_04143_7', 'PM2_5_04143_8',
       'PM2_5_04143_9', 'PM2_5_04143_10', 'PM2_5_04143_11',
       'PM2_5_04143_12', 'PM2_5_04143_13', 'PM2_5_04143_14',
       'PM2_5_04143_15', 'PM2_5_04143_16', 'PM2_5_04143_17',
       'PM2_5_04143_18', 'PM2_5_04143_19', 'PM2_5_04143_20',
       'PM2_5_04143_21', 'PM2_5_04143_22', 'PM2_5_04143_23',
       'PM2_5_04143_24', 'PM10_04143_1', 'PM10_04143_2', 'PM10_04143_3',
       'PM10_04143_4', 'PM10_04143_5', 'PM10_04143_6', 'PM10_04143_7',
       'PM10_04143_8', 'PM10_04143_9', 'PM10_04143_10', 'PM10_04143_11',
       'PM10_04143_12', 'PM10_04143_13', 'PM10_04143_14', 'PM10_04143_15',
       'PM10_04143_16', 'PM10_04143_17', 'PM10_04143_18', 'PM10_04143_19',
       'PM10_04143_20', 'PM10_04143_21', 'PM10_04143_22', 'PM10_04143_23',
       'PM10_04143_24', 'O3_04143_1', 'O3_04143_2', 'O3_04143_3',
       'O3_04143_4', 'O3_04143_5', 'O3

12000